In [1]:
from pymongo import MongoClient
from datetime import datetime, timezone, timedelta
from pprint import pprint
import pandas as pd
from bson.son import SON
import json
from bson import ObjectId
from bson.json_util import dumps
import time
import pymongo
import gridfs
import logging
import gc

### 1. 创建数据库和表
如果不存在，则创建，否则切换到指定位置。
https://blog.csdn.net/yisun123456/article/details/78591255

In [4]:
client = MongoClient("mongodb://192.168.5.106:27017/")
# db = client.autoops
# db = client['admin']
# db.add_user('root', '123456', roles=[{
#     'role':'userAdminAnyDatabase',
#     'db':'admin',
# }])
# db.authenticate(name="autoops", password="autoops")
# db.createUser(
#   {
#     user: "root",
#     pwd: "123456",
#     roles: [ { role: "userAdminAnyDatabase", db: "admin" } ]
#   }
# )
# db.authenticate(name="root", password="123456")
# db.command("show user")

db = client["autoops_cycle"]
# db.add_user('autoops', 'autoops', roles=[{
#     'role':'readWrite',
#     'db':'autoops_cycle',
# }])

# db.getUsers()
# r = db["system.users"].find()
# list(r)

db.authenticate(name="autoops", password="autoops")

# db.dropDatabase()
# collection = db.train_result
# dir(db)
collection = db['datastream_datas']
# collection.drop()

In [ ]:
df = pd.read_csv("0718_1.csv")
data_entities = df.to_dict(orient='records')
result = collection.insert_many(data_entities)

del df


In [1]:
# { field: { $exists: <boolean> } }
# collection.drop()
# import gc
# print(dir(gc))
# import os
r = collection.find({"datastreamId": 8}, {"_id": 0})
data = list(r)

df = pd.DataFrame(data)
print(df.head())
# print(dir(r))
# print(dir(client))

NameError: name 'collection' is not defined

In [9]:
df.to_pickle("data.pkl")

In [3]:
# gc.get_objects()
gc.get_threshold()

(700, 10, 10)

In [4]:
df = pd.DataFrame(tmp_data.copy())

In [5]:
del tmp_data

In [12]:
# print(tmp[0])
del tmp
del result
# import sys
# tmp_div = 1024*1024*8
# print(sys.getsizeof(result)/tmp_div, "MB")
# print(sys.getsizeof(tmp)/tmp_div, "MB")

In [3]:
tmp = {'Unnamed: 0': 0, 'back_flow': 1.5, 'batch_name': '165041A01-N14-P2', 'create_time': '2019-07-11', 'cycle': 24.33, 'cycle_count': 557290, 'extrude_torque': 8.9, 'id': 254626545, 'inject_pressure': 1.2, 'insert_time': '2019-07-15 14:49:18', 'machine_no': '3FB24', 'screw_pos': 29.5, 'screw_velocity': 1.5, 'start_time': '2019-07-11 05:39:29.000', 'start_time_source': '2019-07-11 05:39:29', 'time_stamp': 0, 'unixtime_stamp': 1562794769000}

data = []
index = 1000000
while index > 0:
    tmp["index"] = index
    data.append(tmp.copy())
    index -= 1
print(len(data))
# json.loads()
# del tmp_data
# del df

1000000


### 2. 数据写入
使用字典格式

In [2]:
collection = db.datastream_datas
# collection.drop()
def query_latest_and_earliest_time_of_data(fact_id):
    r = collection.find({"datastreamId": 85}).limit(100)
#     pipeline = [
#         {"$match": {"factId": {"$eq": fact_id}}},
#         {"$group": {"_id": None, "startTime": {"$min": "$startTime"}, "endTime": {"$max": "$endTime"}}},
#         {"$project": {"_id": 0}},
#     ]
#     r = collection.aggregate(pipeline)
#     print(list(r))
    r = list(r)

    return r

r = query_latest_and_earliest_time_of_data(1)
pprint(r)

NameError: name 'db' is not defined

In [6]:
# collection.drop()  # 测试用，先清空表内容

# 准备数据
time_format = "%Y-%m-%d %H:%M:%S.%f"
time_zone = 8


def strtime_2_utctimestamp(datetime_str):
    datetime_obj = datetime.strptime(datetime_str, time_format)
    datetime_utc = datetime_obj.replace(tzinfo=timezone(timedelta(hours=time_zone)))
    return datetime_utc.timestamp()


df = pd.read_csv("./data/facts1.csv")  # 读入原始数据

# #  根据keyword 过滤数据
# df = df.loc[df['keyword'].isin(keyword_list)]

# 将时间列的时间表示为数值型的时间戳
df['time'] = df['time'].apply(strtime_2_utctimestamp)
df['end'] = df['end'].apply(strtime_2_utctimestamp)

# print(df[time_end_col_name][0])
fact_datas_list = []
num = 0
for index, row in df.iterrows():
    fact_data_dict = {
        "factId": 64,
        "startTime": datetime.fromtimestamp(row["time"], tz=timezone.utc),
        "endTime": datetime.fromtimestamp(row["end"], tz=timezone.utc),
        "entity": row["person"],
        "value": row["activity"],
        "keyword": row["keyword"],
        "additionalKeyword": "",
    }
    fact_datas_list.append(fact_data_dict)
#     if num == 2:
#         break
#     num += 1
# pprint(fact_datas_list)

In [7]:
# collection.drop()
def save_test_datas(data: list):
    result = collection.insert_many(data)
    print(dir(result))
    return len(result.inserted_ids)

save_test_datas(fact_datas_list)

['_InsertManyResult__acknowledged', '_InsertManyResult__inserted_ids', '_WriteResult__acknowledged', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_raise_if_unacknowledged', 'acknowledged', 'inserted_ids']


39

In [6]:
# collection = db.datastream_datas
# collection.drop()
def query_all():
#     condition = [{'datastreamId': 1}, {'_id': {'$where': 'this._id % 30 > 0'}}, {'_id': {'$where': 'this._id % 30 <= 15'}}]
#     result = collection.find({'datastreamId': 44}, {'_id': 0,'haha': "$_id"}).limit(1)
    
    time = datetime.fromtimestamp(1491925753, tz=timezone.utc)
    
    print(dir(timezone))
    print(time)
    print(type(time))
    print(datetime(2017, 4, 11, 15, 47, 33, 469000).replace(tzinfo=timezone.utc).timestamp())
#     time = datetime(time)
#     print(time)
    result = collection.find({"factId": {"$lte":65}, "startTime": {"$lte": time}},{"_id":0}).limit(5)
    return list(result)

r = query_all()
pprint(r)

# print("hah{}".format(1))


['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getinitargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'dst', 'fromutc', 'max', 'min', 'tzname', 'utc', 'utcoffset']
2017-04-11 15:49:13+00:00
<class 'datetime.datetime'>
1491925653.469
[]


## 3. 查询

In [5]:
# collection.drop()  # 测试用，先清空表内容
# save_test_datas(fact_datas_list)
def query_all():
    result = collection.find()
    return list(result)

r = query_all()
pprint(r)

def query_fact_datas():
    result = collection.find({"factId": {"$exists": True}}, {"factId": 1, "_id": 0}).sort(
        [("_id", pymongo.DESCENDING)]).limit(4).skip(2)
#     result = collection.find({"$or": [{'factId': 1}, {'keyword': {"$eq": "Training"}}]})
    return list(result)

# r = query_fact_datas()
# pprint(r)

[]


In [6]:
collection.drop()  # 测试用，先清空表内容
# 查询特殊行The following query matches only those documents where the results array contains at least one element that is both greater than or equal to 80 and is less than 85.
def query_fact_datas():
#     result = collection.find({"$and": [{ 'results': { '$elemMatch': { '$gte': 80, '$lt': 85 } }},{"_id": 2}]},)
    result = collection.find({ 'results': { '$elemMatch': { '$gte': 76, '$lte': 82 } } })
    return list(result)

test_data = [
    { '_id': 1, 'results': [ 85, 88, 82 ] },
    { '_id': 2, 'results': [ 75, 88, 89 ] }]


save_test_datas(test_data)

r = query_fact_datas()
pprint(r)

[{'_id': 1, 'results': [85, 88, 82]}]


In [22]:
collection.drop()  # 测试用，先清空表内容
# 查询特殊行
def query_fact_datas():
    result = collection.find({ 'results': { '$elemMatch': { "product": {"$regex" :"xyz*"}} }})
    return list(result)

test_data = [
    { '_id': 1, "name": "haha", 'results': [ { 'product': "abc", 'score': 10 }, { 'product': "xyz", 'score': 5 } ] },
    { '_id': 2, "name": "haha", 'results': [ { 'product': "abc", 'score': 8 }, { 'product': "xyz", 'score': 7 } ] },
    { '_id': 3, "name": "heihei", 'results': [ { 'product': "abc", 'score': 7 }, { 'product': "xyz", 'score': 8 } ] }]


save_test_datas(test_data)

r = query_fact_datas()
pprint(r)

[{'_id': 1,
  'name': 'haha',
  'results': [{'product': 'abc', 'score': 10}, {'product': 'xyz', 'score': 5}]},
 {'_id': 2,
  'name': 'haha',
  'results': [{'product': 'abc', 'score': 8}, {'product': 'xyz', 'score': 7}]},
 {'_id': 3,
  'name': 'heihei',
  'results': [{'product': 'abc', 'score': 7}, {'product': 'xyz', 'score': 8}]}]


## 4. 删除数据

In [36]:
def delete_fact_datas():
    result = collection.delete_many({"factId": 11})
    # result = collection.delete_one({"factId": 1})
#     collection.drop()
    return result.deleted_count
    
r = delete_fact_datas()
print(r)

39


## 5. 更新数据

In [3]:
def update_fact_datas():
    modify = {"$set": {"heihei": 2, "haha": 2}}
    # modify = {"$set": {"entity": 2}}
#     result = collection.update_one({"_id": {"$exist": True}}, modify)
    result = collection.update_many({"_id": {"$eq": 1}}, modify)
    return result

update_fact_datas()
r = query_all()
pprint(r)

print("-------------------------------")

def update_fact_datas():
    modify = {"$unset": {"heihei": 2}}
    # modify = {"$set": {"entity": 2}}
#     result = collection.update_one({"_id": {"$exist": True}}, modify)
    result = collection.update_many({"_id": {"$eq": 1}}, modify)
    return result

update_fact_datas()
r = query_all()
pprint(r)

print("-------------------------------")

def update_fact_datas():
    modify = {
        "id": 3,
        "name": '12'
    }
    result = collection.replace_one({"_id": {"$gt": 2}}, modify)
    return result

update_fact_datas()
r = query_all()
pprint(r)

NameError: name 'collection' is not defined

## 6. 聚合

In [16]:
def count_fact_datas():
    pipeline = [
#         {"$unwind": "$results"},
#         {"$match": {"factId": {"$gt": 5}}},
#         {"$match": {"factId": {"$mod": [2, {"$eq": 1}]}}},
        { "$project" : {"_id": 1, "haha": "$_id"}},
#         {"$group": {"_id": "$factId",  "num": {"$sum": 1}}},
#         {'$sample': {'size': 30}},
#         {"$sort": SON([("_id", 1), ("count", 1)])},
        {"$limit": 1},
    ]
    r = collection.aggregate(pipeline)
    return list(r)

r = count_fact_datas()
pprint(r)

[{'_id': ObjectId('5d076c96c676ed4535a99926'),
  'haha': ObjectId('5d076c96c676ed4535a99926')}]


## 7. 大文件存储gridfsDb

In [4]:
def find_files():
    gridfsDb = client.gridfs
    someFile = gridfsDb.fs.files.find_one({"file_name": "green_alpha_1x1.png"})
    
    print(type(gridfsDb.fs.files))
    print(dir(gridfsDb.fs.files))
    # gridfsDb = client.gridfs
    fs = gridfs.GridFS(gridfsDb)
    print(someFile["_id"])
    tmp_file = fs.get(someFile["_id"])

    # print(dir(tmp_file))
    # print(tmp_file.read())
    # print(type(tmp_file))
    with open('myfile.png', 'wb') as f:
        f.write(tmp_file.read())
    # gridfsDb.fs.download_to_stream(someFile, file)
    return someFile


def file_save():
    gridfsDb = client.gridfs
    # print(dir(gridfsDb.fs.files))
    gridfsDb.fs.files.remove()
    fs = gridfs.GridFS(gridfsDb)
    with open("./data/green_alpha_1x1.png", "rb") as f:
        a = fs.put(f.read(), file_name="green_alpha_1x1.png")
    # a = fs.put(f)
    print(a)

file_save()

r = find_files()
pprint(r)


NameError: name 'client' is not defined